In [4]:
import requests
import sys
import json
from pathlib import Path

In [5]:
# VLLM 서버 주소
llama_api_url = "http://114.70.193.164:8001/v1/completions"

# 입력 및 출력 파일
input_file = Path("data/K2-00001-CL33762-CP33206-05-08-S2.json")
output_file = Path("outputs/augmented/K2-00001-augmented.json")

# 증강 주제
topic = "편지"


In [6]:
with open(input_file, "r", encoding="utf-8") as f:
    raw_data = json.load(f)


In [7]:
def build_prompt(data, topic):
    persona1 = data["participantsInfo"]["speaker1"]
    persona2 = data["participantsInfo"]["speaker2"]
    utterances = data["utterances"]

    history = ""
    for utt in utterances:
        speaker = "A" if utt["speaker"] == "speaker1" else "B"
        history += f"{speaker}: {utt['utterance']}\n"

    prompt = f"""You are a Korean conversation assistant.
Your job is to continue the following two-person conversation on the topic '{topic}'.

Persona A: {persona1}
Persona B: {persona2}

Conversation so far:
{history}
Please add 3~4 more turns in Korean that naturally continue the conversation.
"""
    return prompt


In [8]:
prompt_input = build_prompt(raw_data, topic)

final_input = (
    f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a helpful assistant."
    f"<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt_input}<|eot_id|>"
    f"<|start_header_id|>assistant<|end_header_id|>"
)


KeyError: 'utterances'

In [ ]:
headers = {"Content-Type": "application/json"}

payload = {
    "model": "ibnzterrell/Meta-Llama-3.3-70B-Instruct-AWQ-INT4",
    "prompt": final_input,
    "max_tokens": 512
}

response = requests.post(llama_api_url, json=payload, headers=headers)
response.status_code, response.text[:300]  # 응답 상태 확인


In [ ]:
if response.status_code == 200:
    result_text = response.json()["choices"][0]["text"]
    print("🔁 생성된 응답:\n", result_text)

    # 결과를 기존 JSON에 추가
    raw_data["augmented_response"] = result_text

    # 디렉토리 생성 후 저장
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(raw_data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ 저장 완료: {output_file}")
else:
    print("❌ 요청 실패:", response.status_code)
